In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from random import randint
from random import random
from time import sleep
import pandas as pd

In [2]:
chrome_driver = 'C:/Users/vishw/Downloads/chromedriver_win32/chromedriver'

chrome_options = webdriver.ChromeOptions()
#headless means running chrome with chrome.exe
chrome_options.add_argument('--headless')

chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

## Links scraping

In [ ]:
attractions_driver = webdriver.Chrome(chrome_driver,options=chrome_options)

In [ ]:
links = pd.DataFrame(columns=['Place', 'Link'])

In [ ]:
import unicodedata
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [ ]:
# attractions_driver.refresh()
next_link = 'https://www.tripadvisor.com/Attractions-g4-Activities-a_allAttractions.true-Europe.html'
attractions_driver.get(next_link)
# attractions_driver.get('https://www.tripadvisor.com/Attractions-g4-Activities-a_allAttractions.true-Europe.html')
pg_cnt = 0
while pg_cnt < 1000:
    sleep(randint(4,7) + random())
    attractions = attractions_driver.find_elements(By.CSS_SELECTOR, 'section[data-automation="WebPresentation_SingleFlexCardSection"]')
#     print(len(attractions))
#     while len(attractions) == 0:
#         attractions_driver.find_element(By.CLASS_NAME, 'rmyCe _G B- z _S c Wc wSSLS AeLHi sOtnj').click()
#         attractions_driver.get(next_link)
#         attractions = attractions_driver.find_elements(By.CSS_SELECTOR, 'section[data-automation="WebPresentation_SingleFlexCardSection"]')
    for attraction in attractions:
        sleep(random())
        place = strip_accents(attraction.find_element(By.CLASS_NAME, 'VLKGO').find_element(By.TAG_NAME, 'a').text)
        link = attraction.find_element(By.CLASS_NAME, 'VLKGO').find_element(By.TAG_NAME, 'a').get_attribute('href')
        item = pd.DataFrame([[place, link]], columns=['Place', 'Link'])
        links = pd.concat([links, item], axis = 0, ignore_index = True)
    pg_cnt += 30
#     print(pg_cnt)
    next_link = 'https://www.tripadvisor.com/Attractions-g4-Activities-oa'+str(pg_cnt)+'-Europe.html'
    sleep(randint(2,5) + random())
    attractions_driver.get(next_link)
    attractions_driver.refresh()
    print(len(links), 'links scraped')

In [ ]:
links
# next_link

In [ ]:
links.to_csv('TripAdvisor Links.csv')

## Reviews scraping

In [3]:
review_links_df = pd.read_csv('TripAdvisor Links.csv', index_col = 'Unnamed: 0')
review_links_df

,Place,Link
0,1. Tower of London,https://www.tripadvisor.com/Attraction_Review-...
1,2. Musee d'Orsay,https://www.tripadvisor.com/Attraction_Review-...
2,3. St. Peter's Basilica,https://www.tripadvisor.com/Attraction_Review-...
3,4. Rijksmuseum,https://www.tripadvisor.com/Attraction_Review-...
4,5. Basilica de la Sagrada Familia,https://www.tripadvisor.com/Attraction_Review-...
...,...,...
1015,1016. Gamle Stavanger,https://www.tripadvisor.com/Attraction_Review-...
1016,1017. Hoo Zoo and Dinosaur World,https://www.tripadvisor.com/Attraction_Review-...
1017,1018. Area Archeologica di Libarna,https://www.tripadvisor.com/Attraction_Review-...
1018,1019. Cais da Ribeira,https://www.tripadvisor.com/Attraction_Review-...


In [4]:
review_driver = webdriver.Chrome(chrome_driver,options=chrome_options)

C:\Users\vishw\AppData\Local\Temp\ipykernel_24420\2916931836.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  review_driver = webdriver.Chrome(chrome_driver,options=chrome_options)


In [5]:
all_reviews = pd.DataFrame(columns=['Place','Country', 'City', 'Description', 'Price', 'Rating', 'Popular Mentions', 'Review'])

In [29]:
for index, review_link in review_links_df[209:].iterrows():
    review_driver.get(review_link['Link'])
    sleep(randint(10,15) + random())
    review_driver.refresh()
    place = review_link['Place']
    country = [i.text for i in review_driver.find_elements(By.CSS_SELECTOR,\
                                                       'a[class="BMQDV _F G- wSSLS SwZTJ EabUM FGwzt ukgoS"]')][1]
    city = [i.text for i in review_driver.find_elements(By.CSS_SELECTOR,\
                                                        'a[class="BMQDV _F G- wSSLS SwZTJ EabUM FGwzt ukgoS"]')][-2]
    try:
        description = review_driver.find_element(By.CSS_SELECTOR, 'div[class="_d MJ"]').find_element(By.TAG_NAME, 'div').\
                                                            find_element(By.TAG_NAME, 'div').text
    except:
        description = 'N/A'
    try:
        price = review_driver.find_element(By.CSS_SELECTOR, 'div[class="biGQs _P fiohW uuBRH"]').text[5:]
    except:
        price = 0
    rating = review_driver.find_element(By.CSS_SELECTOR, 'div[class="biGQs _P fiohW hzzSG uuBRH"]').text
    
    try:
        pop_mentions = review_driver.find_element(By.CSS_SELECTOR, 'div[class="qWPrE XCaFq bSHRx"]').text.split('\n')
    except:
        pop_mentions = 'N/A'
    
    ind_reviews = pd.DataFrame(columns=['Place','Country', 'City', 'Description', 'Price', 'Rating', 'Popular Mentions', 'Review'])
    while len(ind_reviews) < 100:
#         sleep(randint(4,7) + random())
        sleep(random())
        try:
            revs = review_driver.find_elements(By.CSS_SELECTOR, 'div[class="fIrGe _T bgMZj"]')[3:]
            for rev in revs:
                review = rev.text
                item = pd.DataFrame([[place, country, city, description, price, rating, pop_mentions, review]],\
                                    columns=ind_reviews.columns)
                ind_reviews = pd.concat([ind_reviews, item], axis = 0, ignore_index = True)
            next_page = review_driver.find_element(By.CSS_SELECTOR, \
                                                   'a[class="BrOJk u j z _F wSSLS tIqAi unMkR"][data-smoke-attr="pagination-next-arrow"]'\
                                                   ).get_attribute('href')
            sleep(randint(1,3) + random())
            review_driver.get(next_page)
            review_driver.refresh()
        except:
            break
    all_reviews = pd.concat([all_reviews, ind_reviews], axis = 0, ignore_index = True)
    print(review_link['Place'], 'scraped. # of reviews scraped:',len(all_reviews))


210. Mundomar scraped. # of reviews scraped: 20655
211. Historical Museum of the Miramare Castle scraped. # of reviews scraped: 20763
212. National Gallery scraped. # of reviews scraped: 20865
213. Ortigia scraped. # of reviews scraped: 20973
214. Royal Yacht Britannia scraped. # of reviews scraped: 21092
215. River Avon scraped. # of reviews scraped: 21192
216. Vittoriosa Waterfront - Birgu Waterfront scraped. # of reviews scraped: 21252
217. Aiguille du Midi scraped. # of reviews scraped: 21355
218. Ancient Theatre of Taormina scraped. # of reviews scraped: 21463
219. Piazza Maggiore scraped. # of reviews scraped: 21571
220. Seven Hanging Valleys Trail scraped. # of reviews scraped: 21579
221. Il Sentiero Degli Dei scraped. # of reviews scraped: 21683
222. Lighthouse Egersheld scraped. # of reviews scraped: 21731
223. The Rock of Gibraltar scraped. # of reviews scraped: 21839
224. Monasterio de Piedra scraped. # of reviews scraped: 21947
225. Plaka Forest scraped. # of reviews scrape

KeyboardInterrupt: 

In [30]:
all_reviews

,Place,Country,City,Description,Price,Rating,Popular Mentions,Review
0,1. Tower of London,United Kingdom (UK),London,"Discover London’s castle – a secure fortress, ...",$36.78,4.5,"[beefeater tour, crown jewels are, yeoman ward...",We purchased tickets in advance for the openin...
1,1. Tower of London,United Kingdom (UK),London,"Discover London’s castle – a secure fortress, ...",$36.78,4.5,"[beefeater tour, crown jewels are, yeoman ward...",I have lived in London almost 10 years and hav...
2,1. Tower of London,United Kingdom (UK),London,"Discover London’s castle – a secure fortress, ...",$36.78,4.5,"[beefeater tour, crown jewels are, yeoman ward...",A very enjoyable day out at the Tower of Londo...
3,1. Tower of London,United Kingdom (UK),London,"Discover London’s castle – a secure fortress, ...",$36.78,4.5,"[beefeater tour, crown jewels are, yeoman ward...",I really feel like they sell too many tickets ...
4,1. Tower of London,United Kingdom (UK),London,"Discover London’s castle – a secure fortress, ...",$36.78,4.5,"[beefeater tour, crown jewels are, yeoman ward...",Fantastic tour of the Tower of London made spe...
...,...,...,...,...,...,...,...,...
30125,307. Ilica Plaji,Turkey,Cesme,... probably has the warmest waters in Turkey ...,0,4.0,"[beautiful beach, public beach, turkey, turkis...",We came here early morning and it was a lovely...
30126,307. Ilica Plaji,Turkey,Cesme,... probably has the warmest waters in Turkey ...,0,4.0,"[beautiful beach, public beach, turkey, turkis...","great for kids, shallow water for about 10 yar..."
30127,307. Ilica Plaji,Turkey,Cesme,... probably has the warmest waters in Turkey ...,0,4.0,"[beautiful beach, public beach, turkey, turkis...","Great beach, 1,7 km long! Keep in mind that it..."
30128,307. Ilica Plaji,Turkey,Cesme,... probably has the warmest waters in Turkey ...,0,4.0,"[beautiful beach, public beach, turkey, turkis...","This is a great beach, and the sand was great!..."


In [31]:
all_reviews.to_csv('all_reviews.csv')

In [25]:
review_links_df[185:195]

,Place,Link
185,186. Sea Organ (Morske Orgulje),https://www.tripadvisor.com/Attraction_Review-...
186,187. Southern Gothenburg Archipelago,https://www.tripadvisor.com/Attraction_Review-...
187,188. Muzeum Oscypka Zakopane,https://www.tripadvisor.com/Attraction_Review-...
188,189. Zoo Leipzig,https://www.tripadvisor.com/Attraction_Review-...
189,190. Fort George,https://www.tripadvisor.com/Attraction_Review-...
190,191. Animaris Ilha Deserta,https://www.tripadvisor.com/Attraction_Review-...
191,192. Old Town,https://www.tripadvisor.com/Attraction_Review-...
192,193. Jardin des Plantes,https://www.tripadvisor.com/Attraction_Review-...
193,194. Illuminated Fountain,https://www.tripadvisor.com/Attraction_Review-...
194,195. Reserva Natural Especial de Las Dunas de ...,https://www.tripadvisor.com/Attraction_Review-...


In [ ]:
country = [i.text for i in review_driver.find_elements(By.CSS_SELECTOR,\
                                                       'a[class="BMQDV _F G- wSSLS SwZTJ EabUM FGwzt ukgoS"]')][1]
city = [i.text for i in review_driver.find_elements(By.CSS_SELECTOR,\
                                                    'a[class="BMQDV _F G- wSSLS SwZTJ EabUM FGwzt ukgoS"]')][-2]
description = review_driver.find_element(By.CSS_SELECTOR, 'div[class="_d MJ"]').find_element(By.TAG_NAME, 'div').\
                                                    find_element(By.TAG_NAME, 'div').text
price = review_driver.find_element(By.CSS_SELECTOR, 'div[class="biGQs _P fiohW uuBRH"]').text[5:]
rating = review_driver.find_element(By.CSS_SELECTOR, 'div[class="biGQs _P fiohW hzzSG uuBRH"]').text
pop_mentions = review_driver.find_element(By.CSS_SELECTOR, 'div[class="qWPrE XCaFq bSHRx"]').text.split('\n')
while len(reviews) < 100:
    sleep(randint(4,7) + random())
    revs = review_driver.find_elements(By.CSS_SELECTOR, 'div[class="fIrGe _T bgMZj"]')[3:]
    for rev in revs:
        review = rev.text
        item = pd.DataFrame([[country, city, description, price, rating, pop_mentions, review]],\
                            columns=reviews.columns)
        reviews = pd.concat([reviews, item], axis = 0, ignore_index = True)
    next_page = review_driver.find_element(By.CSS_SELECTOR, \
                                           'a[class="BrOJk u j z _F wSSLS tIqAi unMkR"][data-smoke-attr="pagination-next-arrow"]'\
                                           ).get_attribute('href')
    print(next_page)
    sleep(randint(2,5) + random())
    review_driver.get(next_page)
    review_driver.refresh()
#     print(len(reviews), 'reviews scraped')

In [ ]:
reviews

In [ ]:
next_page

In [ ]:
# [i.text for i in review_driver.find_elements(By.CSS_SELECTOR, 'span[class="biGQs _P pZUbB avBIb osNWb"]')]
[i.text for i in review_driver.find_elements(By.CSS_SELECTOR, 'a[class="BMQDV _F G- wSSLS SwZTJ EabUM FGwzt ukgoS"]')][1]

In [ ]:
[i.text for i in review_driver.find_elements(By.CSS_SELECTOR, 'a[class="BMQDV _F G- wSSLS SwZTJ EabUM FGwzt ukgoS"]')][-2]

In [ ]:
review_driver.find_element(By.CSS_SELECTOR, 'div[class="_d MJ"]').find_element(By.TAG_NAME, 'div').find_element(By.TAG_NAME, 'div').text

In [ ]:
review_driver.find_element(By.CSS_SELECTOR, 'div[class="biGQs _P fiohW uuBRH"]').text[5:]

In [ ]:
review_driver.find_element(By.CSS_SELECTOR, 'div[class="qWPrE XCaFq bSHRx"]').text.split('\n')

In [ ]:
review_driver.find_element(By.CSS_SELECTOR, 'div[class="biGQs _P fiohW hzzSG uuBRH"]').text

In [ ]:
review_driver.find_element(By.CSS_SELECTOR, 'div[class="biGQs _P fiohW qWPrE ncFvv fOtGX"]').text

In [ ]:
# review_driver.find_elements(By.CSS_SELECTOR, 'div[class="fIrGe _T bgMZj"]')[3].text
[i.text for i in review_driver.find_elements(By.CSS_SELECTOR, 'div[class="fIrGe _T bgMZj"]')[3:]]

In [ ]:
review_driver.find_element(By.CSS_SELECTOR, 'div[class="UCacc"]').find_element(By.TAG_NAME, 'a').get_attribute('href')

In [ ]:
attractions_driver.get('https://www.tripadvisor.com/Attractions-g4-Activities-a_allAttractions.true-Europe.html')
# attractions_driver.get('https://www.tripadvisor.com/Attractions-g4-Activities-oa240-Europe.html')


In [ ]:
attractions = attractions_driver.find_elements(By.CSS_SELECTOR, 'section[data-automation="WebPresentation_SingleFlexCardSection"]')

In [ ]:
len(attractions)

In [ ]:
attractions[0].find_elements(By.CLASS_NAME, 'VLKGO')[0].find_element(By.TAG_NAME, 'a').get_attribute('href')

In [ ]:
attractions[0].find_elements(By.CLASS_NAME, 'VLKGO')[0].find_element(By.TAG_NAME, 'a').text

In [ ]:
next_page = attractions_driver.find_element(By.CLASS_NAME, 'xkSty')
# len(next_page)

In [ ]:
next_page.find_element(By.TAG_NAME, 'a').get_attribute('href')

In [ ]:
attractions = attractions_driver.find_elements(By.CSS_SELECTOR, 'section[data-automation="WebPresentation_SingleFlexCardSection"]')

In [ ]:
# attractions_driver.get('https://www.tripadvisor.com/Attractions-g4-Activities-a_allAttractions.true-Europe.html')
attractions = attractions_driver.find_elements(By.CSS_SELECTOR, 'section[data-automation="WebPresentation_SingleFlexCardSection"]')
print(len(attractions))
for attraction in attractions:
    place = attraction.find_element(By.CLASS_NAME, 'VLKGO').find_element(By.TAG_NAME, 'a').text
    link = attraction.find_element(By.CLASS_NAME, 'VLKGO').find_element(By.TAG_NAME, 'a').get_attribute('href')
    item = pd.DataFrame([[place, link]], columns=['Place', 'Link'])
    links = pd.concat([links, item], axis = 0, ignore_index = True)
links